In [1]:
import time
import os
import math
import pandas as pd
import numpy as np
import sys

sys.path.append("../utils/")
import date_util
import token_util
import enter_util
import data_util
import plot_util

### 提取数据

In [3]:
# trade_date = "20190830"
# delta_price = 30.0
# stock_list = data_util.get_stock_list(trade_date, delta_price)

### 选出符合入场点的股票

In [ ]:
date = '20190801'
test_choose_list = enter_util.get_condidate_stock(date)

In [ ]:
test_choose_list

In [ ]:
choose_data = enter_util.enter_for_plot(test_choose_list)

In [ ]:
enter_true, enter_false, enter_discount = plot_util.plot_enter(choose_data)

In [ ]:
enter_discount

### 模拟交易

In [17]:
test = pd.read_csv("../data_pulled/day/000001.SZ.csv")

In [20]:
test.columns.to_list

<bound method IndexOpsMixin.tolist of Index(['ts_code', 'trade_date', 'open', 'high', 'low', 'close', 'pre_close',
       'change', 'pct_chg', 'vol', 'amount', 'diff', 'dea', 'macd', 'ema12',
       'ema26'],
      dtype='object')>

In [24]:
class Deal:
    def __init__(self, stock_code=None, enter_date=None, enter_price=None):
        
        self.stock_code = stock_code
        self.state = "hold"
        self.enter_date = enter_date
        self.enter_price = enter_price
        self.out_date = None
        self.out_price = None
        
        self.columns = ['stock_code', 'trade_date', 'open', 'high', 'low', 'close', 'pre_close', 
                        'change', 'pct_chg', 'vol', 'amount', 'diff', 'dea', 'macd', 'ema12', 'ema26']
        self.stock_log = pd.DataFrame(columns = self.columns)
        
    def update_log(self, data):
        if self.stock_log['trade_date'].isin(data["trade_date"]).values.sum() == 0:
            self.stock_log = self.stock_log.append(data).reset_index(drop=True)

In [12]:
state = ["hold", "selled"]
current_deal = [] # 记录正在进行的交易
past_deal = [] # 记录已完成的交易
all_log_columns = ['stock_code', 'name', 'state', 'enter_date', 'hold_days', 'out_date', 'buy_price',
           'sell_price', 'pct_chg', 'pct_chg_each_day', 'amount', 'profit']
all_stock_log = pd.DataFrame(columns = all_log_columns) # 记录每支股票的交易

In [ ]:
# data_pulled/day/里面的数据, type(trade_date) = numpy.int64

In [ ]:
start_date = '20190801'
end_date = '20190901'

ROOT_PATH = os.path.join("..", "data_pulled")
ROOT_PATH = os.path.join(ROOT_PATH, "day")
stock_csv_list = os.listdir(ROOT_PATH)

work_date_list = date_util.get_work_day_range(start_date, end_date)

for date in work_date_list:
    
    if len(current_deal) != 0:
        for deal in current_deal:
            # 更新持仓的数据
            
            stock_data = pd.read_csv(os.path.join(ROOT_PATH, deal.stock_code + ".csv"))
            data_today = stock_data[stock_data["trade_date"]==deal.enter_date]
            deal.update_log(data_today)
        
            # 出场(t+1)天
            
        
        
    # 选股
    
    choose_list = enter_util.get_condidate_stock(date)
    stock_choosed = choose_list_tmp.iloc[0]
    
    # 进场(t+1天)
    
    stock_code = stock_choosed["stock_code"]
    enter_date = int(date) + 1
    
    stock_path = os.path.join(ROOT_PATH, stock_choosed["stock_code"] + ".csv")
    file_exist = os.path.exists(stock_path)
    if file_exist:
        stock_data = pd.read_csv(stock_path)
        data_enter = stock_data[stock_data["trade_date"]==enter_date]
    
    deal = Deal(stock_code = stock_code,
                enter_date = enter_date,
                enter_price = data_enter['open']
               )
    deal.update_log(data_enter)
    current_deal.append(deal)
    
    tmp_log = pd.DataFrame([[
        stock_code, stock_choosed["name"], state[0], enter_date, "", "", data_enter['open'],
        "", "", "", 100, ""
    ]], columns = all_log_columns)
    all_stock_log = all_stock_log.append(tmp_log)
    